# data cleaning

In [22]:
import pandas as pd
import os
from urllib.parse import urlparse, parse_qs
import glob
import polars as pl
from polars import col
from zipfile import ZipFile, ZIP_DEFLATED
import os
import datetime

In [23]:
# Directory containing the CSV files
csv_directory = "2023-05-18"

## Get the data

In [24]:
# A list to hold all the data
data = []

# Iterate over each file in the directory
for filename in os.listdir(csv_directory):
    # Check if the file is a CSV file
    if filename.endswith(".csv"):
        # Construct the full file path
        full_filepath = os.path.join(csv_directory, filename)
        
        # Load the data from the CSV file
        df = pl.read_csv(full_filepath, separator=";")
        
        # Add a column to track the original file
        df = df.with_columns(pl.lit(filename).alias('original_file'))
        
        # Add the data to our list
        data.append(df)

# Combine all the data into one DataFrame
all_data = pl.concat(data)

# Remove duplicate rows
all_data = all_data.unique(keep='first', subset=all_data.columns[:-1])

# polars dataframe to pandas dataframe
df_original = all_data.to_pandas()

## clean the data

In [25]:
df_clean = df_original.copy()

In [26]:
# reset the index
df_clean.reset_index(drop=True, inplace=True)

# remove duplicated columns
df_clean = df_original.drop(columns=['Episode_URL_level2',
                                        'Episode_Name_level2',
                                        'Episode_Likes_level2',
                                        'Episode_Date_level2',
                                        'Comment_Reply_Level_level1',
                                        'Comment_Reply_Level_level2',
                                        'Comment_Parent_Comment_No_level1',])

# rename columns
df_clean.rename(columns={'Episode_URL_level1': 'Episode_URL',
                            'Episode_Name_level1': 'Episode_Name',
                            'Episode_Likes_level1': 'Episode_Likes',
                            'Episode_Date_level1': 'Episode_Date',
                            'Comment_ID_level1': 'Comment_ID',
                            'Comment_Text_level1': 'Comment_Text',
                            'Comment_Date_level1': 'Comment_Date',
                            'Comment_Author_level1': 'Comment_Author',
                            'Comment_Likes_level1': 'Comment_Likes',
                            'Comment_Dislikes_level1': 'Comment_Dislikes',
                            'Comment_ID_level2': 'Reply_ID',
                            'Comment_Parent_Comment_No_level2': 'Reply_to_Comment_ID',
                            'Comment_Text_level2': 'Reply_Text',
                            'Comment_Date_level2': 'Reply_Date',
                            'Comment_Author_level2': 'Reply_Author',
                            'Comment_Likes_level2': 'Reply_Likes',
                            'Comment_Dislikes_level2': 'Reply_Dislikes',}, inplace=True)

## Series / Episodes
### Episode ID, series name

In [27]:
def parse_url(url):
    # get genre, series name, episode name
    path = urlparse(url).path
    components = path.split('/')
    genre_generic = components[2]
    series_name = components[3]
    episode_name = components[4]
    # get id
    parsed_url = urlparse(url)
    query_params = parse_qs(parsed_url.query)
    episode_number = query_params.get('episode_no', [None])[0]
    return genre_generic, series_name, episode_name, episode_number

# apply parse_url function to Episode_URL column
df_clean['parsed_url'] = df_clean['Episode_URL'].apply(parse_url)

In [28]:
# Episode ID
df_clean['episode_ID_number'] = df_clean['parsed_url'].apply(lambda x: x[3])

# Series name
df_clean['series_name'] = df_clean['parsed_url'].apply(lambda x: x[1])
df_clean['series_name'] = df_clean['series_name'].str.title().str.replace('-', ' ')

### Series genre, views, subscribers, rating

In [29]:
genre_mapping = {
    'weakhero': 'Action',
    'power-pills': 'Superhero/Comedy',
    'a-life-through-selfies': 'Thriller/Crime/Mystery',
    'how-to-be-a-dragon': 'Fantasy/Comedy',
    'nerd-and-jock': 'Comedy/Slice of life',
    'goth-girl-the-jock': 'Romance/Drama',
    'seekers-log': 'Sci-fi/Drama',
    'my-husband-changes-every-night': 'Romance'
}

views_mapping = {
    'weakhero': 216_300_000,
    'power-pills': 12_500_000,
    'a-life-through-selfies': 41_800_000,
    'how-to-be-a-dragon': 4_900_000,
    'nerd-and-jock': 74_900_000,
    'goth-girl-the-jock': 31_200_000,
    'seekers-log': 3_100_000,
    'my-husband-changes-every-night': 4_500_000
}

subscribers_mapping = {
    'weakhero': 1_400_000,
    'power-pills': 57_465,
    'a-life-through-selfies': 129_226,
    'how-to-be-a-dragon': 112_907,
    'nerd-and-jock': 420_816,
    'goth-girl-the-jock': 643_531,
    'seekers-log': 35_517,
    'my-husband-changes-every-night': 408_319
}

rating_mapping = {
    'weakhero': 9.84,
    'power-pills': 8.57,
    'a-life-through-selfies': 7.12,
    'how-to-be-a-dragon': 9.84,
    'nerd-and-jock': 9.73,
    'goth-girl-the-jock': 9.51,
    'seekers-log': 9.55,
    'my-husband-changes-every-night': 9.32
}

# Define functions to extract values from the mappings using the series_name
def genre_webtoons(parsed_url):
    series_name = parsed_url[1]
    return genre_mapping.get(series_name)

def series_views(parsed_url):
    series_name = parsed_url[1]
    return views_mapping.get(series_name)

def series_subscribers(parsed_url):
    series_name = parsed_url[1]
    return subscribers_mapping.get(series_name)

def series_rating(parsed_url):
    series_name = parsed_url[1]
    return rating_mapping.get(series_name)

# Create new columns using the mapping functions
df_clean['series_genre'] = df_clean['parsed_url'].apply(genre_webtoons)
df_clean['series_views'] = df_clean['parsed_url'].apply(series_views)
df_clean['series_subscribers'] = df_clean['parsed_url'].apply(series_subscribers)
df_clean['series_rating'] = df_clean['parsed_url'].apply(series_rating)

## dates

In [30]:
# change dates to datetime
df_clean['Episode_Date'] = pd.to_datetime(df_clean['Episode_Date'], format='%b %d, %Y')
df_clean['Comment_Date'] = pd.to_datetime(df_clean['Comment_Date'], format='ISO8601')
df_clean['Reply_Date'] = pd.to_datetime(df_clean['Reply_Date'], format='ISO8601')

## Comments

In [31]:
# Remove \n in Comment_Text and Reply_Text
df_clean['Comment_Text'] = df_clean['Comment_Text'].str.replace('\n', ' ')
df_clean['Reply_Text'] = df_clean['Reply_Text'].str.replace('\n', ' ')

## Reorder columns

In [32]:
df_clean = df_clean[['series_genre',
                    'series_name',
                    'series_views',
                    'series_subscribers',
                    'series_rating',
                    'episode_ID_number',
                    'Episode_Name',
                    'Episode_URL',
                    'Episode_Date',
                    'Episode_Likes',
                    'Comment_ID',
                    'Comment_Text',
                    'Comment_Date',
                    'Comment_Author',
                    'Comment_Likes',
                    'Comment_Dislikes',
                    'Reply_ID',
                    'Reply_to_Comment_ID',
                    'Reply_Text',
                    'Reply_Date',
                    'Reply_Author',
                    'Reply_Likes',
                    'Reply_Dislikes']]

## Save csv

In [33]:
# save the dataframe as a csv file like comments_ + todays date and hour + _clean.csv
todaydate = pd.to_datetime('today').strftime('%Y-%m-%d_%H%M')
df_clean_csv_path = 'comments_' + todaydate + '_clean.csv'

df_clean.to_csv(df_clean_csv_path, sep=';', index=False, encoding='utf-8-sig')

zipfile = df_clean_csv_path + '.zip'
with ZipFile(zipfile, 'w', compression=ZIP_DEFLATED, compresslevel=6) as zip:
    zip.write(df_clean_csv_path)

# delete the csv file
os.remove(df_clean_csv_path)

In [34]:
df_clean

,series_genre,series_name,series_views,series_subscribers,series_rating,episode_ID_number,Episode_Name,Episode_URL,Episode_Date,Episode_Likes,...,Comment_Author,Comment_Likes,Comment_Dislikes,Reply_ID,Reply_to_Comment_ID,Reply_Text,Reply_Date,Reply_Author,Reply_Likes,Reply_Dislikes
0,Action,Weakhero,216300000,1400000,9.84,100,Ep. 100,https://www.webtoons.com/en/action/weakhero/ep...,2020-09-01,81653,...,WillowLikesToDraw,49374,38,132769823.0,129697318.0,same,2020-09-01 12:15:41+09:00,FruitInABasket,11.0,0.0
1,Action,Weakhero,216300000,1400000,9.84,100,Ep. 100,https://www.webtoons.com/en/action/weakhero/ep...,2020-09-01,81653,...,TRUEHAWAIIAN,2236,5,129711567.0,129700425.0,nah that's not a weapon it's the clip Stephen ...,2020-08-11 10:54:49+09:00,Jamrar,262.0,1.0
2,Action,Weakhero,216300000,1400000,9.84,100,Ep. 100,https://www.webtoons.com/en/action/weakhero/ep...,2020-09-01,81653,...,Finanov,1414,2,132775449.0,129698199.0,Is no one going to talk about the fact that th...,2020-09-01 13:01:39+09:00,•○♧YeongHooky♧○•,83.0,0.0
3,Action,Weakhero,216300000,1400000,9.84,100,Ep. 100,https://www.webtoons.com/en/action/weakhero/ep...,2020-09-01,81653,...,manas42069,115,0,129734450.0,129717025.0,dropped out because he couldn't handle hearing...,2020-08-11 13:32:07+09:00,spanglepants,49.0,0.0
4,Action,Weakhero,216300000,1400000,9.84,100,Ep. 100,https://www.webtoons.com/en/action/weakhero/ep...,2020-09-01,81653,...,JarOfCuteKats,74,0,132771056.0,132030398.0,O he'll we r gonna,2020-09-01 12:25:30+09:00,mïźů ðä møøñ,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
433865,Romance,My Husband Changes Every Night,4500000,408319,9.32,5,Episode 5,https://www.webtoons.com/en/fantasy/my-husband...,2023-05-15,24878,...,Carrie Rock,8,0,NaN,NaN,None,NaT,None,NaN,NaN
433866,Romance,My Husband Changes Every Night,4500000,408319,9.32,5,Episode 5,https://www.webtoons.com/en/fantasy/my-husband...,2023-05-15,24878,...,GymnastForLife,4,0,NaN,NaN,None,NaT,None,NaN,NaN
433867,Romance,My Husband Changes Every Night,4500000,408319,9.32,5,Episode 5,https://www.webtoons.com/en/fantasy/my-husband...,2023-05-15,24878,...,johannesmuffins,2,0,NaN,NaN,None,NaT,None,NaN,NaN
433868,Romance,My Husband Changes Every Night,4500000,408319,9.32,5,Episode 5,https://www.webtoons.com/en/fantasy/my-husband...,2023-05-15,24878,...,McGill07,1,0,NaN,NaN,None,NaT,None,NaN,NaN
